# Sistemas Lineares, Erros e Aritmética de Pontos Flutuantes


Nesse notebook será desenvolvido o projeto que irá responder os desafios propostos na Atividade 01 de MS211 - Cálculo Numérico

## Instalação de Bibliotecas


In [1]:
%pip install numpy
%pip install scipy
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Importando bibliotecas

In [2]:
import numpy as np
import scipy
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt

### Questão 1

Considerando o sistema linear $Ax = b$, em que
$$A = \begin{bmatrix} 0.1 & 0.2 & 0.3 \\ 0.4 & 0.5 & 0.6 \\ 0.7 & 0.8 & 0.9 \end{bmatrix}, \quad b = \begin{bmatrix} 0.1 \\ 0.3 \\ 0.5 \end{bmatrix}$$

In [3]:
A = np.array([[0.1, 0.2, 0.3], 
              [0.4, 0.5, 0.6], 
              [0.7, 0.8, 0.9]])
b = np.array([[0.1],[0.3],[0.5]])

Uma matriz é dita singular quando ela não possui inversa, isso também pode ser interpretado como o determinante dessa matriz sendo igual a 0. <br>
Abaixo, vemos que o determinante é muito próximo de 0.

In [4]:
det_A = np.linalg.det(A)
det_A

6.661338147750926e-18

#### Item A)

Como o cálculo do determinante de uma matriz envolve produtos de pivôs, primeiro vamos alterar essa matriz para que não aconteçam erros devido a pontos flutuantes.<br>
Sabe-se que se uma matriz B puder ser escrita como B = $k^n$*A, com k sendo escalar, n sendo a dimensão da matriz e A uma matriz de mesma dimensão que B, então o determinante da matriz B será

$$ det(B) = k^n*det(A) $$

Vamos dizer que a matriz B auxiliar será B = 10*A


In [5]:
B = 10*A
B

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

Daí então, calcula-se o determinante da matriz B

In [6]:
det_B = np.linalg.det(B)
det_B

0.0

Ou seja, det(B) = 0 = $10^3$*det(A), logo det(A) = 0. <br>
Como a matriz A é singular, o sistema pode ser inconsistente, ou consistente com infinitas soluções. Para analisar isso, vemos o posto das duas matrizes.

In [7]:
rank_A = np.linalg.matrix_rank(A)
Ab = np.hstack((A, b.reshape(-1, 1)))
rank_Ab = np.linalg.matrix_rank(Ab)
if rank_Ab == rank_A:
    print("Sistema Consistente")
else:
    print("Sistema Inconsistente")

Sistema Consistente


Como o posto de A é igual ao posto da matriz aumentada $A | b$, então o sistema é consistente, logo possui infinitas soluções.<br>
O conjunto solução é do tipo 
$$ x = x_p + t v, \quad t \in \mathbb{R} $$

In [8]:
sol_particular, _, _, _ = np.linalg.lstsq(B, b, rcond=None)
nucleo = scipy.linalg.null_space(B)
sol_particular, nucleo

(array([[0.03888889],
        [0.02222222],
        [0.00555556]]),
 array([[-0.40824829],
        [ 0.81649658],
        [-0.40824829]]))

Nesse caso "sol_particular" é o $xp$ e "nucleo" é a base ortonormal $v$ para o espaço A

#### Item B)


Com aritmética exata, o método falha quando o último pivô é 0.<br>
Vamos verificar os pivôs da decomposição LU

In [9]:
P, L, U = scipy.linalg.lu(A)
np.diag(U)

array([7.00000000e-01, 8.57142857e-02, 1.11022302e-16])

Fazendo eliminação direta em aritmética exata sobre a matriz inteira 𝐵 (proporcional a 𝐴): <br>
Pivot $𝑎_{11} = 1$. Eliminar abaixo:<br>
$𝑅_2 ← 𝑅_2 − 4*𝑅_1 ⇒ [0, −3, −6]$<br>
$𝑅_3 ← 𝑅_3 − 7*𝑅_1 ⇒ [0, −6, −12]$<br>
Pivot na segunda etapa é −3 (não zero). Eliminar abaixo:<br>
$𝑅_3 ← 𝑅_3 −2*𝑅_2 ⇒ [0, 0, 0]$

Agora restou uma linha zero; ao tentar usar essa linha como terceiro pivot, ele seria 0 — não há pivô e o processo não consegue obter uma matriz triangular com pivôs não-nulos para voltar a uma solução única. Portanto não é possível avançar na eliminação ao tentar obter o terceiro pivô (o sistema é singular), o que indica número infinito de soluções (ou inconsistência, se a linha zero viesse acompanhada de termo independente não-zero).<br>

Com pivoteamento parcial não se evita essa situação porque as linhas são proporcionais de modo que o terceiro pivô será zero: não há um elemento não nulo útil para o terceiro pivô.

#### Item C)

Em representações de ponto-flutuante (base 2), números como 0.1 não são exatamente representáveis. Portanto a matriz 𝐴 armazenada no computador não é exatamente $\frac{1}{10}*𝐵$ — ela é uma aproximação. Assim, numericamente 𝐴 fica quase singular, mas não exatamente singular. 

In [10]:
x_num = np.linalg.solve(A,b)
x_num

array([[ 0.1672619 ],
       [ 0.66547619],
       [-0.16607143]])

Como visto acima, métodos diretos podem retornar uma solução numérica porque a matriz representada tem um pequeno desvio que torna o determinante numérico diferente de zero. Entretanto a matriz é extremamente mal condicionada: isso significa que pequenas perturbações (por exemplo erros de arredondamento) podem causar grandes mudanças na solução.


In [11]:
sol_aprox, _, _, _ = np.linalg.lstsq(A,b, rcond=None)
sol_aprox

array([[0.38888889],
       [0.22222222],
       [0.05555556]])

No nosso cálculo com NumPy obtivemos que:

- np.linalg.solve(A,b) retornou uma solução por causa da não-exatidão de 𝐴 em ponto-flutuante.
- O método dos mínimos quadrados utilizando o pseudoinverso de A (np.linalg.lstsq) deu uma solução particular exata (em termos racionais), retornando a solução de norma mínima: $$ 𝑥_𝑝 = [\frac{7}{18}, \frac{2}{9}, \frac{1}{18}]^𝑇$$

As duas soluções diferem por um múltiplo do vetor do núcleo, isto é, ambas são soluções da equação numericamente, e a diferença entre elas é compatível com $$ 𝑡*[−1, 2, −1] $$

Resumindo: o método numérico não "falha" porque o computador trabalha com uma versão levemente perturbada de 𝐴, mas os resultados devem ser tratados com cautela por causa da altíssima sensibilidade.

#### Item D)

O número de condição mede o quão sensível é o sistema a pequenas perturbações nos dados. 

In [12]:
cond_A = np.linalg.cond(A)
cond_A

2.37588029981422e+16

Em ponto-flutuante double (numa máquina com $𝜀_{mach} ≈ 2.22*10^{−16}), a amplificação de erro típica é da ordem de: $$ cond(𝐴)*𝜀_{mach} $$
Nesse caso<br> $$ cond(𝐴)*𝜀_{mach} ≈ 2.3758*10^{16}*2.22*10^{−16} ≈ 5.27 > 1 $$<br>

Ou seja, qualquer erro de arredondamento pode gerar um erro relativo na solução da ordem de unidade — portanto não se pode confiar em dígitos significativos da solução obtida numericamente.<br>
Uma estimação grosseira do número de dígitos decimais corretos é $$ log_{10}(cond(𝐴)) $$<br>
O que dá um número negativo. Em termos práticos isso significa 0 dígitos confiáveis.

Para a matriz exata 𝐴 (matematicamente) o condicionamento é infinito, uma vez que det(𝐴) = 0, o que implica que a matriz não é invertível. Numericamente obtemos um valor muito grande mas finito por causa da representação em ponto flutuante.

### Questão 2

#### Item A)
Em cada junta a treliça está em equilíbrio estático, ou seja a soma vetorial das forças (componentes horizontal e vertical) que atuam naquela junta deve ser zero. Para cada junta escrevemos as duas equações (horizontal e vertical) — algumas juntas têm apenas uma equação independente. Projetando forças nas direções 𝑥(horizontal) e 𝑦(vertical) obtemos equações lineares em $$ 𝑓_1, … , 𝑓_{13} $$

A montagem do sistema é exatamente transcrever essas equações em forma linear $$ 𝐴𝑓 = 𝑏 $$
A convenção utilizada foi que o somatório das componentes vetoriais = 0, que “para a direita” em 𝑥 e que “para cima” em 𝑦 como positivas.

As equações são:

- Junta 2 <br>
$𝑓_2 − 𝑓_6 = 0$ (equilíbrio horizontal na junta 2) <br>
$𝑓_3 = 10$ (equilíbrio vertical — a barra vertical 3 sustenta a carga 10)

- Junta 3<br>
$𝛼*𝑓_1 − 𝑓_4 − 𝛼*𝑓_5 = 0$ (equilíbrio horizontal em 3) <br>
$𝛼*𝑓_1 + 𝑓_3 + 𝛼*𝑓_5 = 0$ (equilíbrio vertical em 3)

- Junta 4<br>
$𝑓_4 − 𝑓_8 = 0$ (horizontal)<br>
$𝑓_7 = 0$ (vertical — a força na barra 7 é zero)

- Junta 5<br>
$𝛼*𝑓_5 + 𝑓_6 − 𝛼*𝑓_9 − 𝑓_{10} = 0$ (horizontal em 5)<br>
$𝛼*𝑓_5 + 𝑓_7 + 𝛼*𝑓_9 = 15$ (vertical em 5 — sustenta carga 15)

- Junta 6<br
$𝑓_{10} − 𝑓_{13} = 0$ (horizontal em 6)<br>
$𝑓_{11} = 20$ (vertical — a barra 11 sustenta a carga 20)

- Junta 7<br>
$𝑓_8 + 𝛼*𝑓_9 − 𝛼*𝑓_{12} = 0 <br>
$𝛼*𝑓_9 + 𝑓_{11} + 𝛼*𝑓_{12} = 0$

- Junta 8<br>
$𝑓_{13} + 𝛼*𝑓_{12} = 0$


Cada uma dessas equações é linear nas incógnitas $𝑓_𝑖$. Agora escrevemos o sistema como $𝐴𝑓 = 𝑏$ tomando as equações na mesma ordem acima.


In [13]:
alpha = np.sqrt(2)/2
n = 13

# Montagem da matriz A e do vetor b (índices: f1->0 ... f13->12)
A = np.zeros((n,n), dtype=float)
b = np.zeros(n, dtype=float)

def set_row(row, coeffs, rhs=0.0):
    for idx, val in coeffs:
        A[row, idx] = val
    b[row] = rhs

set_row(0, [(1, 1.0), (5, -1.0)], 0.0)
set_row(1, [(2, 1.0)], 10.0)
set_row(2, [(0, alpha), (3, -1.0), (4, -alpha)], 0.0)
set_row(3, [(0, alpha), (2, 1.0), (4, alpha)], 0.0)
set_row(4, [(3, 1.0), (7, -1.0)], 0.0)
set_row(5, [(6, 1.0)], 0.0)
set_row(6, [(4, alpha), (5, 1.0), (8, -alpha), (9, -1.0)], 0.0)
set_row(7, [(4, alpha), (6, 1.0), (8, alpha)], 15.0)
set_row(8, [(9, 1.0), (12, -1.0)], 0.0)
set_row(9, [(10, 1.0)], 20.0)
set_row(10, [(7, 1.0), (8, alpha), (11, -alpha)], 0.0)
set_row(11, [(8, alpha), (10, 1.0), (11, alpha)], 0.0)
set_row(12, [(12, 1.0), (11, alpha)], 0.0)

A_sparse = sp.csr_matrix(A)

np.set_printoptions(precision=4, suppress=True)
A, b


(array([[ 0.    ,  1.    ,  0.    ,  0.    ,  0.    , -1.    ,  0.    ,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.7071,  0.    ,  0.    , -1.    , -0.7071,  0.    ,  0.    ,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.7071,  0.    ,  1.    ,  0.    ,  0.7071,  0.    ,  0.    ,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  1.    ,  0.    ,  0.    ,  0.    ,
         -1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  1.    ,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ,  0.7071,  1.    ,  0.    ,
          0.    , -0.7071, -1.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ,  0.

Como 𝐴 é uma matriz quadrada 13×13 e invertível, existe solução única $$ 𝑓 = 𝐴^{−1}*𝑏 $$

Métodos diretos comuns:
- Eliminação de Gauss / fatoração LU com pivotamento parcial: Custo $\mathbb{O}(n^3)$ para matrizes densas. Numericamente robusto e recomendado para sistemas pequenos/medios.

- Solvers esparsos diretos: Usam técnicas de eliminação preservando/evitando preenchimento, vão ajudar nesse caso pois 𝐴 é esparsa.


numpy.linalg.solve chama rotinas LAPACK (LU + pivot) e resolve de forma direta e estável para matrizes densas.<br>

scipy.sparse.linalg.spsolve usa um solver esparso que é eficiente em memória e tempo quando a matriz contém poucos elementos não nulos.<br>
Como 𝐴 tem poucas entradas não-nulas por linha, usar representação esparsa economiza memória e é mais rápido em problemas maiores.

Solução numérica: $$ 𝑓 ≈ [−28.2843, 20, 10, −30, 14.1421, 20, 0, −30, 7.0711, 25, 20, −35.3553, 25]^⊤ $$

O fato de ser obtida uma solução única numericamente indica que 𝐴 é (numericamente) invertível.

In [14]:
# Resolução direta com NumPy (matriz densa)
try:
    f_numpy = np.linalg.solve(A, b)
    print('Solução (NumPy):\n', f_numpy)
except Exception as e:
    print('Erro ao resolver com NumPy:', e)

# Resolução com solver esparso do SciPy
try:
    f_sparse = spla.spsolve(A_sparse, b)
    print('\nSolução (SciPy spsolve):\n', f_sparse)
except Exception as e:
    print('Erro ao resolver esparsamente:', e)

Solução (NumPy):
 [-28.2843  20.      10.     -30.      14.1421  20.       0.     -30.
   7.0711  25.      20.     -35.3553  25.    ]

Solução (SciPy spsolve):
 [-28.2843  20.      10.     -30.      14.1421  20.       0.     -30.
   7.0711  25.      20.     -35.3553  25.    ]


Formulação matricial das iterações:
Escrever 𝐴 = 𝐷 + 𝐿 + 𝑈, onde 𝐷 é a diagonal de 𝐴, 𝐿 é a parte estritamente inferior (triangular inferior sem a diagonal), 𝑈 é a parte estritamente superior.

As formas iterativas são:

- Jacobi: $$ 𝑥^{*𝑘 + 1} = −𝐷^{−1}*(𝐿 + 𝑈)*𝑥^𝑘 + 𝐷^{−1}*𝑏 $$

matriz de iteração $$ 𝐵_J = −𝐷^{−1}*(𝐿 + 𝑈) $$

- Gauss–Seidel: $$ 𝑥^{𝑘 + 1} = −(𝐷 + 𝐿)^{−1}*𝑈*𝑥^k + (𝐷 + 𝐿)^{−1}*𝑏 $$

matriz de iteração $$ 𝐵_{GS} = −(𝐷 + 𝐿)^{−1}*𝑈 $$

Critério de convergência: a iteração converge (para todo $𝑥^{(0)}$) se e somente se 𝜌(𝐵) < 1, onde 𝜌(⋅) é o raio espectral (maior módulo dos autovalores).<br>
Condições suficientes:

- Se 𝐴 é estritamente diagonalmente dominante por linhas $(∣𝑎_{𝑖𝑖}∣ > ∑_{𝑗 ≠ 𝑖} ∣𝑎_{𝑖j}∣$ para todo 𝑖), então tanto Jacobi quanto Gauss–Seidel convergem;

- Se 𝐴 é simétrica definida positiva (SPD), Gauss–Seidel converge; Jacobi pode ou não convergir dependendo da magnitude do raio espectral.

Observando a diagonal de 𝐴 obtida na montagem: muitas entradas da diagonal são zero. 

Na montagem usada os primeiros 11 elementos diagonais são zero e apenas as últimas duas linhas têm diagonal não-nula: isso vem do modo natural como as equações foram colocadas — cada equação foi escrita sem garantir que a incógnita correspondente esteja na diagonal.

Consequências imediatas:

- Jacobi exige $𝐷^{−1}$ — se 𝐷 tem zeros na diagonal, $𝐷^{−1}$ não existe e a fórmula de Jacobi é indeterminada. Assim não se pode aplicar Jacobi diretamente com a ordenação natural das incógnitas/equações usada.

- Gauss–Seidel exige $(𝐷 + 𝐿)^{−1}$. É notável que (D + L) é triangular inferior e sua diagonal é igual à diagonal de 𝐷. Se qualquer elemento da diagonal de 𝐷 é zero, o triangular inferior (𝐷 + 𝐿) tem diagonal zero nessa posição ⇒ (𝐷 + 𝐿) é singular ⇒ não existe $(𝐷 + 𝐿)^{−1}$, logo a fórmula direta de Gauss–Seidel também falha com a ordenação atual. (Isto é exatamente o erro numérico “Singular matrix” que aparece quando tentamos formar a matriz de iteração.)

Portanto, com a modelagem/ordenamento natural utilizada, nem Jacobi nem Gauss–Seidel podem ser aplicados diretamente.


In [16]:
# Análise de convergência para Jacobi e Gauss-Seidel
D = np.diag(np.diag(A))
L = np.tril(A, k=-1)
U = np.triu(A, k=1)

if np.any(np.diag(D) == 0):
    print('A tem zeros na diagonal: Jacobi direto não é aplicável sem rearranjo.')
else:
    Dinv = np.linalg.inv(D)
    B_jacobi = -Dinv.dot(L + U)
    eigs_j = np.linalg.eigvals(B_jacobi)
    rho_j = max(abs(eigs_j))
    print('Raio espectral de Jacobi:', rho_j)

try:
    B_gs = -np.linalg.inv(D + L).dot(U)
    eigs_gs = np.linalg.eigvals(B_gs)
    rho_gs = max(abs(eigs_gs))
    print('Raio espectral de Gauss-Seidel:', rho_gs)
except Exception as e:
    print('Erro ao calcular matriz de iteração de Gauss-Seidel:', e)


A tem zeros na diagonal: Jacobi direto não é aplicável sem rearranjo.
Erro ao calcular matriz de iteração de Gauss-Seidel: Singular matrix
